# Simulating future stock prices using Monte Carlo methods
using Monte Carlo methods, simulate future stock price outcomes for Apple ($APPL) using Python. 

Below is the code using Python to grab the initial stock price data for Apple, to help get you started:
https://www.investopedia.com/terms/m/montecarlosimulation.asp

In [8]:
# Import required libraries
import math
import matplotlib.pyplot as plt
import numpy as np
from pandas_datareader import data
from scipy.stats import norm
apple = data.DataReader('AAPL', 'yahoo',start='1/1/2009')
apple.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-01-02,3.251429,3.041429,3.067143,3.241071,7.460152e+08,2.773207
2009-01-05,3.435000,3.311071,3.327500,3.377857,1.181608e+09,2.890248
2009-01-06,3.470357,3.299643,3.426786,3.322143,1.289310e+09,2.842576
2009-01-07,3.303571,3.223572,3.278929,3.250357,7.530488e+08,2.781153
2009-01-08,3.326786,3.215714,3.229643,3.310714,6.735008e+08,2.832797


In [4]:
# look at the size of the dataset
apple.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2998 entries, 2009-01-02 to 2020-11-27
Data columns (total 6 columns):
High         2998 non-null float64
Low          2998 non-null float64
Open         2998 non-null float64
Close        2998 non-null float64
Volume       2998 non-null float64
Adj Close    2998 non-null float64
dtypes: float64(6)
memory usage: 244.0 KB


Steps to simulate future stock price:
1. Calculate perriodic daily returns: periodic daily return = ln(day's price/previous day's price)
2. Calcualte average, STDEV.P, and VAR.P of periodic daily return
3. Drift = Average Daily Return - Variance/2
4. Obtain random input: Random Value = STDEV * inverse of standard normal cumulative distribution

In [12]:
# calculate periodic daily return
pdr = apple['Close']/apple['Close'].shift(periods=1)
# drop NaN value (first index)
pdr = pdr.dropna()

In [13]:
# calculate avg, standard deviation, and variance
avg = np.mean(pdr)
std = np.std(pdr)
var = np.var(pdr)

In [24]:
drift = avg - (var/2)

In [85]:
# create random value
rand_val = std * norm.ppf(np.random.rand(len(pdr)))

In [88]:
# drop first row in apple dataframe
df = apple.iloc[1:]
# calculate next day stock price
df['prediction'] = [ x * math.exp(drift + y) for x,y in zip(apple['Close'][1:],rand_val)]

In [108]:
# prediction using previous day's calculation instead of all apple close prices
def calc_price():
    rand_val = std * norm.ppf(np.random.rand(len(pdr)))
    start = apple['Close'][1]
    pred = []
    for i in rand_val:
        prediction = start * math.exp(drift + i)
        start = prediction
        pred.append(prediction)

    return pred

In [109]:
df['p2'] = calc_price()

In [110]:
df['p2']

Date
2009-01-05     125.059727
2009-01-06     333.228974
2009-01-07    1026.395093
2009-01-08     981.872471
2009-01-09    3698.182617
                 ...     
2020-11-20            inf
2020-11-23            inf
2020-11-24            inf
2020-11-25            inf
2020-11-27            inf
Name: p2, Length: 2997, dtype: float64

In [102]:
plt.plot(df['Close'])
plt.plot(df['p2'])
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend(['Closing Price','Prediction'])

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000028B342A7048> (for post_execute):


ValueError: arange: cannot compute length

In [97]:
df['p2']

Date
2009-01-05      9.159381
2009-01-06     24.539704
2009-01-07     65.968143
2009-01-08    183.921352
2009-01-09    493.814143
                 ...    
2020-11-20           inf
2020-11-23           inf
2020-11-24           inf
2020-11-25           inf
2020-11-27           inf
Name: p2, Length: 2997, dtype: float64